In [1]:
import anndata
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import scanpy as sc
import sklearn.decomposition #import TruncatedSVD
import scipy 
import sparse, io
import os 
matplotlib.rcParams.update({'font.size': 12})
%config InlineBackend.figure_format = 'retina'

os.chdir('/lunarc/nobackup/projects/regen_immuno/tools/samap/transcriptomes/')

In [36]:
#Load in the data
adata_pl=anndata.read_h5ad('/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/spleen_merged_raw.h5ad')
adata_hum=anndata.read_h5ad('/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Raw_data/Anno/progen.h5ad')

In [10]:
adata_pl.obs

,status,assignment,batch,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,leiden
TCTCACGTCTCCTGAC-1-0,singlet,2,0,1980,1978,69408,287,0.413497,11
TCCCATGGTGGATCAG-1-0,singlet,2,0,2697,2696,63859,191,0.299096,1
GATCAGTAGGTGCTTT-1-0,singlet,1,0,4178,4174,59680,558,0.934987,11
CGATGGCAGAGCTGCA-1-0,singlet,2,0,2370,2369,53256,244,0.458164,11
GTGACGCTCATGCATG-1-0,singlet,1,0,5769,5768,51858,1274,2.456709,11
...,...,...,...,...,...,...,...,...,...
GTGGCGTAGGTTCTAC-1-1,singlet,3,1,498,497,637,36,5.651491,8
ATTCTACCAAGCTACT-1-1,singlet,4,1,467,467,542,1,0.184502,8
AATCGACAGAACGCGT-1-1,singlet,4,1,456,456,562,8,1.423488,0
GTAGTACTCACGAACT-1-1,singlet,4,1,400,400,504,2,0.396825,10


In [37]:
adata_hum.obs

,n_counts,n_genes,mito,predicted_doublets,Sort_id,scvi_clusters,is_maternal_contaminant,celltype_annotation,assay_ontology_term_id,cell_type_ontology_term_id,...,sample_long,self_reported_ethnicity_ontology_term_id,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
index,,,,,,,,,,,,,,,,,,,,,
FCAImmP7579224-TCTTTCCCAAGCCGTC,28616,5176,0.035994,False,CD45P,17,False,PRE_PRO_B,EFO:0011025,CL:0002045,...,F45_SK_CD45P_FCAImmP7579224,unknown,fraction A pre-pro B cell,10x 5' v1,normal,Homo sapiens,female,skin of body,unknown,13th week post-fertilization human stage
FCAImmP7579224-GGAATAATCAGGTAAA,20510,4007,0.044174,False,CD45P,8,False,PRE_PRO_B,EFO:0011025,CL:0002045,...,F45_SK_CD45P_FCAImmP7579224,unknown,fraction A pre-pro B cell,10x 5' v1,normal,Homo sapiens,female,skin of body,unknown,13th week post-fertilization human stage
FCAImmP7579224-TAGCCGGGTACCATCA,18168,3903,0.062087,False,CD45P,13,False,LMPP_MLP,EFO:0011025,CL:0000936,...,F45_SK_CD45P_FCAImmP7579224,unknown,early lymphoid progenitor,10x 5' v1,normal,Homo sapiens,female,skin of body,unknown,13th week post-fertilization human stage
FCAImmP7579224-GCATGCGCAGCTGCAC,14823,3796,0.046414,False,CD45P,11,False,MEMP,EFO:0011025,CL:0000050,...,F45_SK_CD45P_FCAImmP7579224,unknown,megakaryocyte-erythroid progenitor cell,10x 5' v1,normal,Homo sapiens,female,skin of body,unknown,13th week post-fertilization human stage
FCAImmP7579224-GATGCTATCTCTGCTG,12947,3095,0.019927,False,CD45P,8,False,PRE_PRO_B,EFO:0011025,CL:0002045,...,F45_SK_CD45P_FCAImmP7579224,unknown,fraction A pre-pro B cell,10x 5' v1,normal,Homo sapiens,female,skin of body,unknown,13th week post-fertilization human stage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FCAImmP7277565-CGTAGGCAGACAGACC,4003,1460,0.018236,False,CD45N,5,False,DN(P)_T,EFO:0009899,CL:0002489,...,F30_TH_CD45N_FCAImmP7277565,unknown,double negative thymocyte,10x 3' v2,normal,Homo sapiens,male,thymus,unknown,15th week post-fertilization human stage
FCAImmP7277565-GCATGCGGTAGAGCTG,3802,1329,0.018148,False,CD45N,6,False,DN(P)_T,EFO:0009899,CL:0002489,...,F30_TH_CD45N_FCAImmP7277565,unknown,double negative thymocyte,10x 3' v2,normal,Homo sapiens,male,thymus,unknown,15th week post-fertilization human stage
FCAImmP7277565-GTACGTAAGAGGTAGA,3768,1283,0.027070,False,CD45N,6,False,DN(P)_T,EFO:0009899,CL:0002489,...,F30_TH_CD45N_FCAImmP7277565,unknown,double negative thymocyte,10x 3' v2,normal,Homo sapiens,male,thymus,unknown,15th week post-fertilization human stage


In [11]:
#Samap analysis
from samap.mapping import SAMAP
from samap.analysis import (get_mapping_scores, GenePairFinder,
                            sankey_plot, chord_plot, CellTypeTriangles,
                            ParalogSubstitutions, FunctionalEnrichment,
                            convert_eggnog_to_homologs, GeneTriangles)

In [38]:
org1='/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/spleen_merged_raw.h5ad'
org2='/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Raw_data/Anno/progen.h5ad'
filenames = {'pw':org1,'hs':org2}
sm = SAMAP(
        filenames,
        f_maps = 'maps/',
        save_processed=True, #if False, do not save the processed results to `*_pr.h5ad`
        keys = {'pw':'leiden','hs':'cell_type'},
   )
sm.run()
samap = sm.samap

Processing data pw from:
/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/spleen_merged_raw.h5ad
RUNNING SAM
Iteration: 0, Convergence: 1.0
Iteration: 1, Convergence: 0.8635011261046498
Computing the UMAP embedding...


/home/nikvaku/.local/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Elapsed time: 23.967145919799805 seconds
Not updating the manifold...
Processing data hs from:
/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Raw_data/Anno/progen.h5ad
RUNNING SAM
Iteration: 0, Convergence: 1.0
Iteration: 1, Convergence: 0.8839096067386398
Iteration: 2, Convergence: 0.01013068981277829
Computing the UMAP embedding...


/home/nikvaku/.local/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Elapsed time: 88.58161067962646 seconds
Not updating the manifold...
2036 `pw` gene symbols match between the datasets and the BLAST graph.
18569 `hs` gene symbols match between the datasets and the BLAST graph.
Prepping datasets for translation.


...

/home/nikvaku/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/nikvaku/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Prepping datasets for translation.


/home/nikvaku/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/nikvaku/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Translating feature spaces pairwise.
Projecting data into joint latent space. 4.796600818634033
Correcting data with means. 11.299850940704346
Expanding neighbourhoods of species pw...
Expanding neighbourhoods of species hs...
Indegree coarsening
0/3 (0, 55303)
1/3 (20000, 55303)
2/3 (40000, 55303)
Rescaling edge weights by expression correlations.
Concatenating SAM objects...
Iteration 3 complete.
Alignment scores:
         hs        pw
hs  0.00000  0.573479
pw  0.66067  0.000000
Running UMAP on the stitched manifolds.
Elapsed time: 6.13624906539917 minutes.


/home/nikvaku/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [39]:
keys_ct = {'pw':'leiden','hs':'cell_type'}
Ct,MappingTable_ct = get_mapping_scores(sm,keys_ct,n_top = 100)

In [40]:
MappingTable_ct

,pw_0,pw_1,pw_10,pw_11,pw_12,pw_13,pw_14,pw_15,pw_2,pw_3,...,hs_early lymphoid progenitor,hs_fraction A pre-pro B cell,hs_granulocyte monocyte progenitor cell,hs_hematopoietic multipotent progenitor cell,hs_hematopoietic stem cell,hs_megakaryocyte,hs_megakaryocyte-erythroid progenitor cell,hs_native cell,hs_promonocyte,hs_promyelocyte
pw_0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.858578,0.907846,0.524759,0.812766,0.793911,0.533153,0.659531,0.475465,0.327531,0.454378
pw_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.461382,0.808084,0.864115,0.970783,0.963101,0.634761,0.958459,0.403769,0.088351,0.033038
pw_10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.016967,0.145901,0.216924,0.049950,0.091438,0.222700,0.365765,0.029219,0.212445,0.107861
pw_11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.007445,0.143575,0.216239,0.164880,0.147721,0.199405,0.324919,0.033563,0.188873,0.157241
pw_12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.016645,0.030631,0.007256,0.024705,0.012777,0.008045,0.005255,0.024855,0.005982,0.006523
pw_13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001661,0.000000,0.589968,0.000420,0.001025,0.011461,0.000189,0.001461,0.609975,0.479638
pw_14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010189,0.001989,0.000000,0.001994,0.000594,0.002481,0.006617,0.015793,0.000000,0.000000
pw_15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005887,0.005606,0.000435,0.003433,0.049738,0.017942,0.095043,0.005651,0.000000,0.000000
pw_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.087245,0.099935,0.419976,0.167991,0.050220,0.563259,0.455568,0.133419,0.036809,0.149087
pw_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005739,0.004536,0.120140,0.434506,0.140833,0.891751,0.533274,0.104491,0.397493,0.053790


In [41]:
MappingTable_ct.to_csv("/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/Integrated_SAMAP_table_progen.csv")

In [34]:
pd.read_csv("/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/Integrated_SAMAP_table_bonemarrow.csv")

,Unnamed: 0,pw_0,pw_1,pw_10,pw_11,pw_12,pw_13,pw_14,pw_15,pw_2,...,hs_natural killer cell,hs_non-classical monocyte,hs_non-terminally differentiated cell,hs_plasma cell,hs_plasmacytoid dendritic cell,hs_pre-natural killer cell,hs_precursor B cell,hs_pro-B cell,hs_small pre-B-II cell,hs_unswitched memory B cell
0,pw_0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.023662,0.436827,0.0,0.160018,0.033033,0.025120,0.319201,0.244634,0.026738,0.598933
1,pw_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005354,0.017144,0.0,0.416959,0.019944,0.001040,0.103135,0.236459,0.024535,0.626013
2,pw_10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000157,0.037931,0.0,0.039893,0.000999,0.000000,0.000311,0.000000,0.000000,0.153589
3,pw_11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000193,0.018497,0.0,0.011948,0.000835,0.000000,0.005909,0.003764,0.000000,0.128945
4,pw_12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.068464,0.002349,0.0,0.007508,0.000248,0.000000,0.000202,0.000065,0.000000,0.012936
5,pw_13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000497,0.001026,0.0,0.000000,0.349006,0.000000,0.000000,0.000000,0.000000,0.001327
6,pw_14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.079279,0.005734,0.0,0.001081,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,pw_15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007900
8,pw_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.061693,0.667903,0.0,0.021265,0.190399,0.000240,0.000794,0.000485,0.000000,0.015209
9,pw_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.089586,0.052986,0.0,0.009082,0.001842,0.000797,0.002363,0.000732,0.000000,0.013468
